## A minimal logist regression example

In [ ]:
import theano
import theano.tensor as T
import numpy as np
rng = np.random

import matplotlib.pyplot as plt

#rng = np.random.RandomState(0)

ninputs = 100
nfeatures = 784
noutputs = 10

#training rate
tr_rate = 0.1 
training_steps = 100

D = (rng.randn(ninputs, nfeatures), rng.randint(size=ninputs, low=0, high=noutputs))

# Feed-forward
x = T.dmatrix('x')
t = T.ivector('t')

w = theano.shared(rng.randn(nfeatures, noutputs), name="w")
b = theano.shared(np.full(noutputs, 0.01), name="b")
sigma = T.nnet.softmax(T.dot(x,w) + b) 
prediction = T.argmax(sigma, axis=1)   


parameters = [w, b]

xent = -T.mean(T.log(sigma)[T.arange(t.shape[0]), t])          # Cost function
    
cost = xent.mean() + 0.01 * (w ** 2).sum()                     # Regularisation 

gparameters = [T.grad(cost, param) for param in parameters]    # Compute the gradient of the cost 
 
pars = zip(parameters, gparameters)

train = theano.function(
          inputs=[x,t],
          outputs=[xent],
          updates = [(param, param - tr_rate * gparam) for param, gparam in zip(parameters, gparameters)],
          allow_input_downcast = True )


predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    train(D[0], D[1])
  

result = predict(D[0]) - D[1]

N = ninputs

error = 0
for index in range(N):
    if result[index] != 0:
        error += 1
        
correct_guesses = N - error
accuracy = (N - error)*100./N

print
print "correct predictions = %f over %i examples" % (correct_guesses, N)
print "accuracy = %f%%" % accuracy


#p = predict(D[0])  
#plt.plot(D[1], 'ro')
#plt.plot(p, 'bo')
#plt.axis([-1, N+1, -1, 10])

#plt.show()